# Генерация больших данных для нагрузочного тестирования Hadoop

Этот ноутбук генерирует синтетический набор данных заданного размера и сохраняет его как таблицу Hive.

**Способ избежания OOM и больших файлов:**
- Мы используем параметр конфигурации `spark.sql.files.maxRecordsPerFile`.
- Это заставляет Spark автоматически "нарезать" выходные файлы по количеству строк.
- Мы ставим значение **3 000 000 строк**, что для текущей структуры данных примерно соответствует 100-120 МБ.

**Custom OutputFormat:**
- Мы также загружаем скомпилированный JAR (`custom-formats.jar`) с вашим классом `com.example.bigdata.SizeBasedParquetOutputFormat`. Он доступен в classpath, если вы захотите использовать его через RDD API.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, rand, concat, expr, hex
import time
import os

# --- ПАРАМЕТРЫ КОНФИГУРАЦИИ ---
NUM_ROWS = 100_000_000  # Желаемое количество строк
TABLE_NAME = "large_scale_table"
MAX_RECORDS_PER_FILE = 3_000_000 # ~120 MB на файл
CUSTOM_JAR_PATH = "/home/jovyan/work/custom-formats.jar"
# ------------------------------

print(f"Конфигурация: {NUM_ROWS} строк, таблица '{TABLE_NAME}'")
if os.path.exists(CUSTOM_JAR_PATH):
    print(f"Custom JAR найден: {CUSTOM_JAR_PATH}")
else:
    print("Custom JAR не найден! Сначала запустите компиляцию.")

In [ ]:
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("LargeScaleDataGen") \
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.jars", CUSTOM_JAR_PATH) \
    .enableHiveSupport() \
    .config("spark.sql.files.maxRecordsPerFile", str(MAX_RECORDS_PER_FILE)) \
    .getOrCreate()

print("Spark Session создана (Custom JAR загружен).")

In [ ]:
# Генерация данных
print("Формирование DataFrame...")

df = spark.range(0, NUM_ROWS).withColumnRenamed("id", "row_id")

df_with_data = df \
    .withColumn("val_int_1", (rand() * 100).cast("int")) \
    .withColumn("val_int_2", (rand() * 10000).cast("int")) \
    .withColumn("val_double_1", rand()) \
    .withColumn("val_double_2", rand() * 1000.0) \
    .withColumn("category_code", (rand() * 5).cast("int").cast("string")) \
    .withColumn("status", expr("elt(cast(rand()*3 as int) + 1, 'ACTIVE', 'INACTIVE', 'PENDING')")) \
    .withColumn("random_string_1", hex(expr("rand() * 100000").cast("long"))) \
    .withColumn("random_string_2", concat(lit("prefix_"), col("row_id"))) \
    .withColumn("created_date", expr("date_add(current_date(), -cast(rand()*365 as int))")) \
    .withColumn("description", lit("This is a test record for load testing Hadoop cluster capacity"))

print("Схема:")
df_with_data.printSchema()

In [ ]:
start_time = time.time()
print(f"Начинаем запись данных в таблицу {TABLE_NAME}...")

# Используем стандартный writer с параметром maxRecordsPerFile
# Это самый стабильный способ контролировать размер файлов.

df_with_data \
    .write \
    .mode("overwrite") \
    .format("parquet") \
    .saveAsTable(f"default.{TABLE_NAME}")

end_time = time.time()
duration = end_time - start_time

print(f"✓ Готово! Записано {NUM_ROWS} строк.")
print(f"Время выполнения: {duration:.2f} секунд")

In [ ]:
# Проверка результата (имен файлов)
spark.sql(f"SELECT * FROM default.{TABLE_NAME} LIMIT 5").show(truncate=False)

count = spark.sql(f"SELECT count(*) FROM default.{TABLE_NAME}").collect()[0][0]
print(f"Проверка количества строк: {count}")

# Показать количество созданных файлов через hadoop input_file_name()
from pyspark.sql.functions import input_file_name
file_count = spark.table(f"default.{TABLE_NAME}").select(input_file_name()).distinct().count()
print(f"Количество созданных файлов: {file_count}")

In [ ]:
spark.stop()